## Encrypted Linear Regression Tutorial
This Jupyter notebook provides an introduction to using EncryptedLinearRegression from the venumML library, built on top of venumpy.

Note: This is a basic example and might require additional libraries for data manipulation and visualization depending on your specific needs.

In [12]:
import sys
sys.path.append('..')

import venumpy
import numpy as np

### Simple Plaintext Linear Regression with scikit-learn
Before diving into EncryptedLinearRegression, let's explore unencrypted linear regression using scikit-learn (sklearn). Sklearn provides a widely used implementation of linear regression with the LinearRegression class.

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Step 1: Generate Sample Data
X, y = make_regression(n_samples=10, n_features=2, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sk_lr = LinearRegression()
sk_lr.fit(X_train, y_train)

print("Scikit-Learn Coefficients:", sk_lr.coef_)
print("Scikit-Learn Intercept:", sk_lr.intercept_)

# Compare with Scikit-Learn predictions
sk_lr_predictions = sk_lr.predict(X_test)

%timeit sk_lr_predictions
print("Scikit-Learn Predictions:", sk_lr_predictions)

Scikit-Learn Coefficients: [55.58970623 15.4175784 ]
Scikit-Learn Intercept: 0.04137759409549702
6.21 ns ± 0.0105 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
Scikit-Learn Predictions: [-34.44463524  47.28440292]


The steps for plaintext linear regression are:

- We import LinearRegression from sklearn.linear_model.
- We create an instance of LinearRegression and train it on the data using fit(X, y).
- We define new data (X_new) for prediction.
- We use the trained model's predict(X_new) method to get the predicted target value.
- We print the predicted value for the new data point.
- This section demonstrates how to use scikit-learn's LinearRegression for prediction, similar to how we'll use EncryptedLinearRegression in the next sections.

### VenumML EncryptedLinearRegression Class
This class implements a linear regression model with support for encrypted data. It allows you to perform encrypted predictions without revealing the underlying model parameters or the data itself.

#### Encryption Approach:

This class is designed to enable computations over encrypted data by using Fully Homomorphic Encryption (FHE). FHE enables computations on encrypted data, allowing the model to perform linear regression without decrypting the input data.

#### Class Attributes:

context: Venumpy context object used for encryption and decryption. This can be defined as an argument in the constructor or as a class attribute depending on how you want to manage the encryption context for your models.


### Encrypted Linear Regression with venumpy

In [14]:
from venumMLlib.linear_models.regression.linear_regression import EncryptedLinearRegression
from venumMLlib.venum_tools import encrypt_array

# Create venumpy context with 128 bits of security
ctx = venumpy.SecretContext.new_with_security(128)
ctx.precision = 6

In [15]:
X = np.array([[1, 2], [2, 3], [3, 5], [4, 7], [5, 11]])  # Features
y = np.array([2, 4, 5, 4, 5])
       
# 1D array
model = EncryptedLinearRegression(ctx)
model.fit(X, y)
model.encrypt_coefficients(ctx)

In [16]:
# Step 1: Sample Data from previous sklearn is used

# Step 2: Train EncryptedLinearRegression and Scikit-Learn LinearRegression
my_lr = EncryptedLinearRegression(ctx)
my_lr.fit(X_train, y_train)

# Compare the Coefficients and Intercept
print("VENum Linear Regression Coefficients:", my_lr.coef_)
print("VENum Linear Regression Intercept:", my_lr.intercept_)

# Test Inference
my_lr.encrypt_coefficients(ctx)

cipher_X = encrypt_array(X_test,ctx)

# Now cipher_X is a flat list of encrypted features
my_lr_predictions = my_lr.predict(cipher_X,ctx)

# Decrypt predictions
decrypted_predictions = [pred.decrypt() for pred in my_lr_predictions]

# Compare with Scikit-Learn predictions
sk_lr_predictions = sk_lr.predict(X_test)

%timeit my_lr_predictions
# Output comparisons (Note: The decrypted predictions need to be reshaped or processed further to match the format of sk_lr_predictions)
print("Decrypted VENum Predictions:", decrypted_predictions)

VENum Linear Regression Coefficients: [55.58970623 15.4175784 ]
VENum Linear Regression Intercept: 0.04137759409550057
6.33 ns ± 0.0212 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
Decrypted VENum Predictions: [-34.444632527844, 47.284414057984]
